In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv('Resources/charity_data.csv')
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
#  YOUR CODE GOES HERE
application_df = application_df.drop(['EIN', 'NAME'], axis=1)

In [3]:
# Look at APPLICATION_TYPE value counts for binning
#  YOUR CODE GOES HERE
application_count = application_df.APPLICATION_TYPE.value_counts()

In [4]:
# Determine which values to replace if counts are less than ...?
#  YOUR CODE GOES HERE
replace_application = list(application_count[application_count < 500].index)
replace_application
# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Look at CLASSIFICATION value counts for binning
#  YOUR CODE GOES HERE
classification_count = application_df.CLASSIFICATION.value_counts()

In [6]:
# Determine which values to replace if counts are less than ..?
#  YOUR CODE GOES HERE
replace_class = list(classification_count[classification_count < 1000].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [7]:
# Generate our categorical variable lists
#  YOUR CODE GOES HERE
application_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()

In [9]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
#  YOUR CODE GOES HERE
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)

In [10]:
# Merge one-hot encoded features and drop the originals
#  YOUR CODE GOES HERE

application_df = application_df.merge(encode_df, left_index=True,right_index=True)
application_df = application_df.drop(application_cat, 1)

In [11]:
# Split our preprocessed data into our features and target arrays
#  YOUR CODE GOES HERE
y = application_df.IS_SUCCESSFUL.values
X = application_df.drop(columns=["IS_SUCCESSFUL"]).values

# Split the preprocessed data into a training and testing dataset
#  YOUR CODE GOES HERE
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [29]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [30]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  15
hidden_nodes_layer2 = 10
hidden_nodes_layer3 = 5
nn = tf.keras.models.Sequential()

# First hidden layer
#  YOUR CODE GOES HERE
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
#  YOUR CODE GOES HERE
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
#  YOUR CODE GOES HERE
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))


# Output layer
#  YOUR CODE GOES HERE
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_29 (Dense)             (None, 15)                660       
_________________________________________________________________
dense_30 (Dense)             (None, 10)                160       
_________________________________________________________________
dense_31 (Dense)             (None, 5)                 55        
_________________________________________________________________
dense_32 (Dense)             (None, 1)                 6         
Total params: 881
Trainable params: 881
Non-trainable params: 0
_________________________________________________________________


In [31]:
# Compile the model
#  YOUR CODE GOES HERE
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [32]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoint2/",exist_ok=True)
checkpoint_path = "checkpoint2/weights2.{epoch:02d}.hdf5"

# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=100)

In [33]:
# Train the model
#  YOUR CODE GOES HERE
fit_model = nn.fit(X_train_scaled, y_train, epochs=150, callbacks=[cp_callback])

Epoch 1/150
 94/804 [==>...........................] - ETA: 1s - loss: 0.6695 - accuracy: 0.6356
Epoch 00001: saving model to checkpoint2\weights2.01.hdf5
176/804 [=====>........................] - ETA: 1s - loss: 0.6453 - accuracy: 0.6690
Epoch 00001: saving model to checkpoint2\weights2.01.hdf5
291/804 [=========>....................] - ETA: 0s - loss: 0.6222 - accuracy: 0.6873
Epoch 00001: saving model to checkpoint2\weights2.01.hdf5
399/804 [=============>................] - ETA: 0s - loss: 0.6088 - accuracy: 0.6984
Epoch 00001: saving model to checkpoint2\weights2.01.hdf5
484/804 [=================>............] - ETA: 0s - loss: 0.6008 - accuracy: 0.7044
Epoch 00001: saving model to checkpoint2\weights2.01.hdf5
574/804 [====================>.........] - ETA: 0s - loss: 0.5955 - accuracy: 0.7078
Epoch 00001: saving model to checkpoint2\weights2.01.hdf5
677/804 [========================>.....] - ETA: 0s - loss: 0.5921 - accuracy: 0.7097
Epoch 00001: saving model to checkpoint2\weig

551/804 [===================>..........] - ETA: 0s - loss: 0.5431 - accuracy: 0.7391
Epoch 00007: saving model to checkpoint2\weights2.07.hdf5
668/804 [=======================>......] - ETA: 0s - loss: 0.5445 - accuracy: 0.7381
Epoch 00007: saving model to checkpoint2\weights2.07.hdf5
756/804 [===========================>..] - ETA: 0s - loss: 0.5474 - accuracy: 0.7359
Epoch 00007: saving model to checkpoint2\weights2.07.hdf5
804/804 [==============================] - 1s 2ms/step - loss: 0.5486 - accuracy: 0.7348
Epoch 8/150
 71/804 [=>............................] - ETA: 1s - loss: 0.5425 - accuracy: 0.7416
Epoch 00008: saving model to checkpoint2\weights2.08.hdf5
167/804 [=====>........................] - ETA: 0s - loss: 0.5512 - accuracy: 0.7333
Epoch 00008: saving model to checkpoint2\weights2.08.hdf5
260/804 [========>.....................] - ETA: 0s - loss: 0.5451 - accuracy: 0.7371
Epoch 00008: saving model to checkpoint2\weights2.08.hdf5
351/804 [============>.................] 

116/804 [===>..........................] - ETA: 1s - loss: 0.5346 - accuracy: 0.7468
Epoch 00014: saving model to checkpoint2\weights2.14.hdf5
217/804 [=======>......................] - ETA: 1s - loss: 0.5395 - accuracy: 0.7389
Epoch 00014: saving model to checkpoint2\weights2.14.hdf5
320/804 [==========>...................] - ETA: 0s - loss: 0.5406 - accuracy: 0.7373
Epoch 00014: saving model to checkpoint2\weights2.14.hdf5
425/804 [==============>...............] - ETA: 0s - loss: 0.5406 - accuracy: 0.7377
Epoch 00014: saving model to checkpoint2\weights2.14.hdf5
518/804 [==================>...........] - ETA: 0s - loss: 0.5404 - accuracy: 0.7390
Epoch 00014: saving model to checkpoint2\weights2.14.hdf5
618/804 [======================>.......] - ETA: 0s - loss: 0.5406 - accuracy: 0.7393
Epoch 00014: saving model to checkpoint2\weights2.14.hdf5
746/804 [==========================>...] - ETA: 0s - loss: 0.5430 - accuracy: 0.7372
Epoch 00014: saving model to checkpoint2\weights2.14.hdf5

596/804 [=====================>........] - ETA: 0s - loss: 0.5422 - accuracy: 0.7367
Epoch 00020: saving model to checkpoint2\weights2.20.hdf5
691/804 [========================>.....] - ETA: 0s - loss: 0.5430 - accuracy: 0.7366
Epoch 00020: saving model to checkpoint2\weights2.20.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.5442 - accuracy: 0.7353
Epoch 21/150
  1/804 [..............................] - ETA: 3s - loss: 0.4300 - accuracy: 0.8438
Epoch 00021: saving model to checkpoint2\weights2.21.hdf5
120/804 [===>..........................] - ETA: 1s - loss: 0.5450 - accuracy: 0.7352
Epoch 00021: saving model to checkpoint2\weights2.21.hdf5
213/804 [======>.......................] - ETA: 1s - loss: 0.5393 - accuracy: 0.7406
Epoch 00021: saving model to checkpoint2\weights2.21.hdf5
300/804 [==========>...................] - ETA: 0s - loss: 0.5413 - accuracy: 0.7368
Epoch 00021: saving model to checkpoint2\weights2.21.hdf5
399/804 [=============>................]

184/804 [=====>........................] - ETA: 1s - loss: 0.5454 - accuracy: 0.7335
Epoch 00027: saving model to checkpoint2\weights2.27.hdf5
280/804 [=========>....................] - ETA: 0s - loss: 0.5469 - accuracy: 0.7334
Epoch 00027: saving model to checkpoint2\weights2.27.hdf5
373/804 [============>.................] - ETA: 0s - loss: 0.5472 - accuracy: 0.7332
Epoch 00027: saving model to checkpoint2\weights2.27.hdf5
463/804 [================>.............] - ETA: 0s - loss: 0.5449 - accuracy: 0.7343
Epoch 00027: saving model to checkpoint2\weights2.27.hdf5
593/804 [=====================>........] - ETA: 0s - loss: 0.5438 - accuracy: 0.7357
Epoch 00027: saving model to checkpoint2\weights2.27.hdf5
690/804 [========================>.....] - ETA: 0s - loss: 0.5424 - accuracy: 0.7372
Epoch 00027: saving model to checkpoint2\weights2.27.hdf5
784/804 [============================>.] - ETA: 0s - loss: 0.5430 - accuracy: 0.7368
Epoch 00027: saving model to checkpoint2\weights2.27.hdf5

...

605/804 [=====================>........] - ETA: 0s - loss: 0.5377 - accuracy: 0.7405
Epoch 00124: saving model to checkpoint2\weights2.124.hdf5
697/804 [=========================>....] - ETA: 0s - loss: 0.5384 - accuracy: 0.7396
Epoch 00124: saving model to checkpoint2\weights2.124.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.5389 - accuracy: 0.7393
Epoch 125/150
  1/804 [..............................] - ETA: 2s - loss: 0.4860 - accuracy: 0.7188
Epoch 00125: saving model to checkpoint2\weights2.125.hdf5
 82/804 [==>...........................] - ETA: 1s - loss: 0.5305 - accuracy: 0.7416
Epoch 00125: saving model to checkpoint2\weights2.125.hdf5
198/804 [======>.......................] - ETA: 1s - loss: 0.5346 - accuracy: 0.7424
Epoch 00125: saving model to checkpoint2\weights2.125.hdf5
282/804 [=========>....................] - ETA: 0s - loss: 0.5313 - accuracy: 0.7472
Epoch 00125: saving model to checkpoint2\weights2.125.hdf5
392/804 [=============>..........

Epoch 00131: saving model to checkpoint2\weights2.131.hdf5
163/804 [=====>........................] - ETA: 1s - loss: 0.5443 - accuracy: 0.7322
Epoch 00131: saving model to checkpoint2\weights2.131.hdf5
271/804 [=========>....................] - ETA: 1s - loss: 0.5410 - accuracy: 0.7375
Epoch 00131: saving model to checkpoint2\weights2.131.hdf5
358/804 [============>.................] - ETA: 0s - loss: 0.5411 - accuracy: 0.7378
Epoch 00131: saving model to checkpoint2\weights2.131.hdf5
473/804 [================>.............] - ETA: 0s - loss: 0.5400 - accuracy: 0.7381
Epoch 00131: saving model to checkpoint2\weights2.131.hdf5
561/804 [===================>..........] - ETA: 0s - loss: 0.5400 - accuracy: 0.7375
Epoch 00131: saving model to checkpoint2\weights2.131.hdf5
673/804 [========================>.....] - ETA: 0s - loss: 0.5380 - accuracy: 0.7396
Epoch 00131: saving model to checkpoint2\weights2.131.hdf5
778/804 [============================>.] - ETA: 0s - loss: 0.5377 - accuracy:

538/804 [===================>..........] - ETA: 0s - loss: 0.5370 - accuracy: 0.7398
Epoch 00137: saving model to checkpoint2\weights2.137.hdf5
628/804 [======================>.......] - ETA: 0s - loss: 0.5370 - accuracy: 0.7403
Epoch 00137: saving model to checkpoint2\weights2.137.hdf5
728/804 [==========================>...] - ETA: 0s - loss: 0.5383 - accuracy: 0.7397
Epoch 00137: saving model to checkpoint2\weights2.137.hdf5
804/804 [==============================] - 1s 2ms/step - loss: 0.5386 - accuracy: 0.7392
Epoch 138/150
 33/804 [>.............................] - ETA: 1s - loss: 0.5529 - accuracy: 0.7244
Epoch 00138: saving model to checkpoint2\weights2.138.hdf5
127/804 [===>..........................] - ETA: 1s - loss: 0.5277 - accuracy: 0.7475
Epoch 00138: saving model to checkpoint2\weights2.138.hdf5
222/804 [=======>......................] - ETA: 0s - loss: 0.5325 - accuracy: 0.7432
Epoch 00138: saving model to checkpoint2\weights2.138.hdf5
323/804 [===========>............

 97/804 [==>...........................] - ETA: 1s - loss: 0.5230 - accuracy: 0.7545
Epoch 00144: saving model to checkpoint2\weights2.144.hdf5
227/804 [=======>......................] - ETA: 1s - loss: 0.5364 - accuracy: 0.7431
Epoch 00144: saving model to checkpoint2\weights2.144.hdf5
315/804 [==========>...................] - ETA: 0s - loss: 0.5364 - accuracy: 0.7409
Epoch 00144: saving model to checkpoint2\weights2.144.hdf5
408/804 [==============>...............] - ETA: 0s - loss: 0.5364 - accuracy: 0.7406
Epoch 00144: saving model to checkpoint2\weights2.144.hdf5
496/804 [=================>............] - ETA: 0s - loss: 0.5379 - accuracy: 0.7390
Epoch 00144: saving model to checkpoint2\weights2.144.hdf5
597/804 [=====================>........] - ETA: 0s - loss: 0.5388 - accuracy: 0.7385
Epoch 00144: saving model to checkpoint2\weights2.144.hdf5
704/804 [=========================>....] - ETA: 0s - loss: 0.5393 - accuracy: 0.7385
Epoch 00144: saving model to checkpoint2\weights2.1

594/804 [=====================>........] - ETA: 0s - loss: 0.5391 - accuracy: 0.7382
Epoch 00150: saving model to checkpoint2\weights2.150.hdf5
684/804 [========================>.....] - ETA: 0s - loss: 0.5404 - accuracy: 0.7380
Epoch 00150: saving model to checkpoint2\weights2.150.hdf5
772/804 [===========================>..] - ETA: 0s - loss: 0.5395 - accuracy: 0.7387
Epoch 00150: saving model to checkpoint2\weights2.150.hdf5
804/804 [==============================] - 1s 2ms/step - loss: 0.5384 - accuracy: 0.7396


In [34]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5629 - accuracy: 0.7254
Loss: 0.5628679394721985, Accuracy: 0.7253644466400146


In [35]:
# Define the model - deep neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  15
hidden_nodes_layer2 = 10
hidden_nodes_layer3 = 5

nn_new = tf.keras.models.Sequential()

# First hidden layer
nn_new.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="tanh"))

# Second hidden layer
nn_new.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="tanh"))

# Third hidden layer
nn_new.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="tanh"))




# Output layer
nn_new.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the model
nn_new.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Restore the model weights
nn_new.load_weights("checkpoint2/weights2.50.hdf5")

In [36]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5629 - accuracy: 0.7254
Loss: 0.5628679394721985, Accuracy: 0.7253644466400146
